<a href="https://colab.research.google.com/github/VictorIM1/veridion-classifier/blob/main/label_description_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance Label Descriptions & Embedding Workflow
Cod complet pentru:
- Generarea de descrieri pentru etichete din taxonomie
- Salvare incrementală în JSON
- Generare embeddings cu all-MiniLM-L6-v2
- Export CSV pentru utilizare ulterioară


In [1]:
import json
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path


In [2]:
taxonomy_path = Path("insurance_taxonomy.csv")
labels = pd.read_csv(taxonomy_path)["label"].tolist()

gpt_model_name = "gpt2"
gpt_tokenizer = AutoTokenizer.from_pretrained(gpt_model_name)
gpt_model = AutoModelForCausalLM.from_pretrained(gpt_model_name)
gpt_model.eval()
if torch.cuda.is_available():
    gpt_model.to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
def generate_description(label):
    prompt = f"Describe the service called '{label}' in a clear and neutral way."
    inputs = gpt_tokenizer(prompt, return_tensors="pt").to(gpt_model.device)
    outputs = gpt_model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=True,
        top_p=0.95,
        temperature=0.9,
        pad_token_id=gpt_tokenizer.eos_token_id
    )
    result = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result[len(prompt):].strip()


In [4]:
output_path = Path("generated_label_descriptions.json")
descriptions = {}
if output_path.exists():
    descriptions = json.loads(output_path.read_text(encoding="utf-8"))

for idx, label in enumerate(tqdm(labels)):
    if label in descriptions:
        continue
    try:
        descriptions[label] = generate_description(label)
    except Exception as e:
        print(f"Eroare la '{label}': {e}")
        descriptions[label] = ""
    if (idx + 1) % 50 == 0:
        output_path.write_text(json.dumps(descriptions, indent=2, ensure_ascii=False), encoding="utf-8")
        print(f"[SALVAT] {idx + 1} descrieri generate...")
output_path.write_text(json.dumps(descriptions, indent=2, ensure_ascii=False), encoding="utf-8")

 23%|██▎       | 50/220 [03:06<10:49,  3.82s/it]

[SALVAT] 50 descrieri generate...


 45%|████▌     | 100/220 [06:27<08:02,  4.02s/it]

[SALVAT] 100 descrieri generate...


 68%|██████▊   | 150/220 [09:43<04:51,  4.17s/it]

[SALVAT] 150 descrieri generate...


 91%|█████████ | 200/220 [12:57<01:17,  3.86s/it]

[SALVAT] 200 descrieri generate...


100%|██████████| 220/220 [14:15<00:00,  3.89s/it]


66739

In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import json

model = SentenceTransformer("all-MiniLM-L6-v2")

with open("generated_label_descriptions.json", encoding="utf-8") as f:
    descriptions = json.load(f)

labels = list(descriptions.keys())
desc_texts = [descriptions[label] for label in labels]

embeddings = model.encode(desc_texts, show_progress_bar=True)

df_embed = pd.DataFrame(embeddings, index=labels)
df_embed.index.name = "label"
df_embed.to_csv("label_embeddings.csv")
print(" label_embeddings.csv a fost salvat cu succes.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ label_embeddings.csv a fost salvat cu succes.


In [6]:
# @title Textul prestabilit al titlului
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from tqdm import tqdm
import json

# Incarca descrierile generate si embeddings pentru labeluri
with open("generated_label_descriptions.json", encoding="utf-8") as f:
    label_descriptions = json.load(f)

label_embeddings_df = pd.read_csv("label_embeddings.csv", index_col="label")
label_embeddings = torch.tensor(label_embeddings_df.values.astype("float32"))
label_names = label_embeddings_df.index.tolist()

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Incarca companiile
df_comp = pd.read_csv("company_list.csv")
df_comp = df_comp.fillna("")

def build_input_text(row):
    return " ".join([
        row["description"],
        row["business_tags"],
        row["sector"],
        row["category"],
        row["niche"]
    ])

# Embedding pentru companii
company_inputs = df_comp.apply(build_input_text, axis=1).tolist()
company_embeddings = embedding_model.encode(company_inputs, show_progress_bar=True)

# Clasificare
results = []
partial_save_path = "partial_predictions.csv"
final_save_path = "final_predictions.csv"

for i, (embedding, raw_text) in enumerate(tqdm(zip(company_embeddings, company_inputs), total=len(company_inputs))):
    cosine_scores = util.cos_sim(embedding, label_embeddings)[0]
    top_indices = np.argsort(-cosine_scores)[:20]
    candidate_labels = [label_names[idx] for idx in top_indices]

    try:
        zshot_result = zero_shot(raw_text, candidate_labels, multi_label=False)
        top_label = zshot_result["labels"][0]
        score = zshot_result["scores"][0]
    except Exception as e:
        print(f"Eroare zero-shot la index {i}: {e}")
        top_label = ""
        score = 0.0

    results.append({
        "company_index": i,
        "predicted_label": top_label,
        "score": score,
        "candidates": candidate_labels,
        "raw_text": raw_text
    })

    if (i + 1) % 200 == 0:
        pd.DataFrame(results).to_csv(partial_save_path, index=False)
        print(f"Salvate predictii partiale pentru {i + 1} companii...")

pd.DataFrame(results).to_csv(final_save_path, index=False)
print(" Clasificare finalizata. Rezultatele sunt salvate in final_predictions.csv")


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Batches:   0%|          | 0/297 [00:00<?, ?it/s]

  1%|          | 101/9494 [1:50:36<171:27:04, 65.71s/it]


KeyboardInterrupt: 

In [8]:
!jupyter nbconvert --to notebook --inplace --ClearMetadataPreprocessor.enabled=True notebook.ipynb

[NbConvertApp] WARNING | pattern 'notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Ex